In [2]:
import pandas as pd
import matplotlib.pyplot as plt 
import re
import textdistance
import numpy as np
%matplotlib inline

In [125]:
# !pip install textdistance

Looking in indexes: http://as-pypi-gcs


In [ ]:
# ee = pd.read_csv('../../data/processed/elevator_to_line_dir_station.csv')

In [10]:
ee = pd.read_csv('../../data/processed/edgelist_w_pid.csv')

In [11]:
ee = ee[ee.to_type == 'Train']

In [15]:
ee['line'] = [x.split('-')[0] for x in ee.to]
ee['direction'] = [x.split('-')[1] for x in ee.to]

In [16]:
ee[ee.station_name.str.contains('34 St - Penn')]

,station_name,from,from_type,to,to_type,line,direction
820,34 St - Penn,Platform pid_140,Platform,C-north,Train,C,north
821,34 St - Penn,Platform pid_140,Platform,E-north,Train,E,north
828,34 St - Penn,Platform pid_141,Platform,A-north,Train,A,north
829,34 St - Penn,Platform pid_141,Platform,A-south,Train,A,south
831,34 St - Penn,Platform pid_142,Platform,C-south,Train,C,south
832,34 St - Penn,Platform pid_142,Platform,E-south,Train,E,south
838,34 St - Penn,Platform pid_143,Platform,1-south,Train,1,south
840,34 St - Penn,Platform pid_144,Platform,2-north,Train,2,north
841,34 St - Penn,Platform pid_144,Platform,2-south,Train,2,south
842,34 St - Penn,Platform pid_144,Platform,3-north,Train,3,north


In [17]:
ee.direction.unique()

array(['south', 'north', 'manhattan', 'norwood 205 st', 'woodlawn',
       'jamaica', 'flushing main st', 'coney island',
       'flatbush av brooklyn college', 'jamaica center', 'flushing',
       'wakefield 241 st', 'coney island, brighton beach',
       'jamaica, forest hills', 'far rockaway', 'brigton beach',
       'brighton beach', 'east', 'west',
       'brighton beach, coney island, bay ridge'], dtype=object)

In [18]:
routes = pd.read_csv('../../data/raw/google_transit/routes.txt')
trips = pd.read_csv('../../data/raw/google_transit/trips.txt')

In [19]:
trips[trips.route_id == 'W'].head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
16420,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_037300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16421,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_039300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16422,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_040300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16423,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_041300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN
16424,W,BFA19GEN-N093-Weekday-00,BFA19GEN-N093-Weekday-00_042300_N..S72R,Whitehall St - South Ferry,1,NaN,NaN


In [20]:
'''
Subset routes_wkd to contain line, fuzzy match station name (to upper) with stop_name and name_cur
'''

'\nSubset routes_wkd to contain line, fuzzy match station name (to upper) with stop_name and name_cur\n'

In [21]:
stops_new = pd.read_csv('../../data/raw/google_transit/stops.txt')

In [22]:
stops_new.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,101,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,1,NaN
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
2,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
3,103,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,1,NaN
4,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103


In [23]:
stops_new[stops_new.stop_name == '30 Av']

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1314,R04,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,1,NaN
1315,R04N,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,0,R04
1316,R04S,NaN,30 Av,NaN,40.766779,-73.921479,NaN,NaN,0,R04


In [24]:
platforms = stops_new[stops_new.location_type == 0]

In [25]:
platforms.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
1,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
2,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101
4,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103
5,103S,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103
7,104N,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,0,104


In [26]:
platforms[platforms.stop_name.str.contains('Queens')]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
508,718N,NaN,Queensboro Plaza,NaN,40.750582,-73.940202,NaN,NaN,0,718
509,718S,NaN,Queensboro Plaza,NaN,40.750582,-73.940202,NaN,NaN,0,718
694,B04N,NaN,21 St - Queensbridge,NaN,40.754203,-73.942836,NaN,NaN,0,B04
695,B04S,NaN,21 St - Queensbridge,NaN,40.754203,-73.942836,NaN,NaN,0,B04
1006,G21N,NaN,Queens Plaza,NaN,40.748973,-73.937243,NaN,NaN,0,G21
1007,G21S,NaN,Queens Plaza,NaN,40.748973,-73.937243,NaN,NaN,0,G21
1327,R09N,NaN,Queensboro Plaza,NaN,40.750582,-73.940202,NaN,NaN,0,R09
1328,R09S,NaN,Queensboro Plaza,NaN,40.750582,-73.940202,NaN,NaN,0,R09


In [27]:
stop_times = pd.read_csv('../../data/raw/google_transit/stop_times.txt')

In [28]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:06:00,00:06:00,101S,1,NaN,0,0,NaN
1,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:07:30,00:07:30,103S,2,NaN,0,0,NaN
2,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:09:00,00:09:00,104S,3,NaN,0,0,NaN
3,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:10:30,00:10:30,106S,4,NaN,0,0,NaN
4,AFA19GEN-1037-Sunday-00_000600_1..S03R,00:12:00,00:12:00,107S,5,NaN,0,0,NaN


In [29]:
stop_times['line'] = [x[x.find('..')-1:x.find('..')] for x in stop_times.trip_id]

In [30]:
weekday = stop_times[stop_times.trip_id.str.contains('Weekday')]

In [31]:
routes_subset = routes[(routes.route_id.str.len() == 1) & (routes.route_id != 'H')]

In [32]:
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_000600_1..S03R,South Ferry,1,NaN,1..S03R
1,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_002600_1..S03R,South Ferry,1,NaN,1..S03R
2,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_004600_1..S03R,South Ferry,1,NaN,1..S03R
3,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_006600_1..S03R,South Ferry,1,NaN,1..S03R
4,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_007200_1..N03R,Van Cortlandt Park - 242 St,0,NaN,1..N03R


In [33]:
weekday = weekday.merge(trips[trips.route_id.isin(routes_subset.route_id)][['trip_id','route_id']],on="trip_id")

In [34]:
unique_stop_ids = pd.DataFrame(weekday.stop_id.unique(),columns=['stop_id'])

In [35]:
unique_stop_ids['routes_wkd'] = [''.join(weekday[weekday.stop_id == x]['route_id'].unique()) for x in unique_stop_ids.stop_id]

In [36]:
platforms = platforms.merge(unique_stop_ids,on='stop_id')

In [37]:
platforms[platforms.stop_name.str.contains('Jackson')]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,routes_wkd
110,220N,NaN,Jackson Av,NaN,40.816490,-73.907807,NaN,NaN,0,220,25
111,220S,NaN,Jackson Av,NaN,40.816490,-73.907807,NaN,NaN,0,220,25
320,709N,NaN,82 St - Jackson Hts,NaN,40.747659,-73.883697,NaN,NaN,0,709,7
321,709S,NaN,82 St - Jackson Hts,NaN,40.747659,-73.883697,NaN,NaN,0,709,7
342,721N,NaN,Vernon Blvd - Jackson Av,NaN,40.742626,-73.953581,NaN,NaN,0,721,7
343,721S,NaN,Vernon Blvd - Jackson Av,NaN,40.742626,-73.953581,NaN,NaN,0,721,7
652,G14N,NaN,Jackson Hts - Roosevelt Av,NaN,40.746644,-73.891338,NaN,NaN,0,G14,EFMR
653,G14S,NaN,Jackson Hts - Roosevelt Av,NaN,40.746644,-73.891338,NaN,NaN,0,G14,EFMR


In [38]:
platforms.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,routes_wkd
0,101N,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101,1
1,101S,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,0,101,1
2,103N,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103,1
3,103S,NaN,238 St,NaN,40.884667,-73.900870,NaN,NaN,0,103,1
4,104N,NaN,231 St,NaN,40.878856,-73.904834,NaN,NaN,0,104,1


In [39]:
ee['possible_stops'] = ''

In [40]:
for index, row in ee.iterrows():
    if row.possible_stops == '':
        subset = platforms[platforms.routes_wkd.str.contains(row.line)]
        if subset.shape[0] > 0:
            subset_stop_names = pd.DataFrame(subset.stop_name.unique(),columns=['stop_name'])
            name_dist = [max(textdistance.jaro_winkler(row.station_name,y),textdistance.jaccard(row.station_name,y)) for y in subset_stop_names.stop_name]
                
            matched_station_name = subset_stop_names.iloc[np.argmax(name_dist),0]
            matched_stop_ids = subset[subset.stop_name == matched_station_name][['stop_id']]
            score = max(name_dist)
            if score > 0.79:
                ee.loc[index,'possible_stops'] = ', '.join(matched_stop_ids.stop_id)
#                 print('{} : {} - {}'.format(row.station_name,matched_station_name,score))


In [267]:
# for index, row in ee.iterrows():
#     if row.stop_ids == '':
#         subset = platforms[platforms.routes_wkd.str.contains(row.line)]
#         if subset.shape[0] > 0:
#             subset_stop_names = pd.DataFrame(subset.stop_name.unique(),columns=['stop_name'])
#             name_dist = [textdistance.jaro_winkler(row.station,y) for y in subset_stop_names.stop_name]
#             matched_station_name = subset_stop_names.iloc[np.argmax(name_dist),0]
#             matched_stop_ids = subset[subset.stop_name == matched_station_name][['stop_id']]
#             score = max(name_dist)
#             if score > 0.5:
# #                 ee.iloc[index,5] = ', '.join(matched_stop_ids.stop_id)
#                 print('{} : {} - {}'.format(row.station,matched_station_name,score))

Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
Broadway-Lafayette/Bleecker St : Brooklyn Bridge - City Hall - 0.535707502374169
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq - 0.79825
West 4 St - Washington Sq : W 4 St - Wash Sq 

In [41]:
ee[ee.possible_stops == ''].shape[0]/ee.shape[0]

0.02666666666666667

In [44]:
ee[ee.station_name.str.contains('Jackson Hts - Roosevelt Av')]

,station_name,from,from_type,to,to_type,line,direction,possible_stops
1023,Jackson Hts - Roosevelt Av,Platform pid_173,Platform,E-north,Train,E,north,"G14N, G14S"
1024,Jackson Hts - Roosevelt Av,Platform pid_173,Platform,F-north,Train,F,north,"G14N, G14S"
1025,Jackson Hts - Roosevelt Av,Platform pid_173,Platform,M-north,Train,M,north,"G14N, G14S"
1026,Jackson Hts - Roosevelt Av,Platform pid_173,Platform,R-north,Train,R,north,"G14N, G14S"
1028,Jackson Hts - Roosevelt Av,Platform pid_174,Platform,E-south,Train,E,south,"G14N, G14S"
1029,Jackson Hts - Roosevelt Av,Platform pid_174,Platform,F-south,Train,F,south,"G14N, G14S"
1030,Jackson Hts - Roosevelt Av,Platform pid_174,Platform,M-south,Train,M,south,"G14N, G14S"
1031,Jackson Hts - Roosevelt Av,Platform pid_174,Platform,R-south,Train,R,south,"G14N, G14S"


In [42]:
ee[ee.possible_stops == '']

,station_name,from,from_type,to,to_type,line,direction,possible_stops
16,125 St,Platform pid_2,Platform,METRO_NORTH-south,Train,METRO_NORTH,south,
17,125 St,Platform pid_2,Platform,METRO_NORTH-north,Train,METRO_NORTH,north,
506,Forest Hills - 71 Av,Platform pid_92,Platform,LIRR-jamaica,Train,LIRR,jamaica,
514,Forest Hills - 71 Av,Platform pid_93,Platform,LIRR-manhattan,Train,LIRR,manhattan,
652,New Utrecht,Platform pid_115,Platform,D-north,Train,D,north,
654,New Utrecht,Platform pid_116,Platform,D-south,Train,D,south,
727,Sutphin Blvd - Archer Av - JFK Airport,Platform pid_131,Platform,LIRR-south,Train,LIRR,south,
728,Sutphin Blvd - Archer Av - JFK Airport,Platform pid_131,Platform,LIRR-north,Train,LIRR,north,
744,Times Sq - 42 St,Platform pid_132,Platform,S-north,Train,S,north,
753,Times Sq - 42 St,Platform pid_132,Platform,S-south,Train,S,south,


In [45]:
ee.loc[ee[(ee.possible_stops == '') & (ee.station_name == 'Broadway-Lafayette/Bleecker St')].index,'possible_stops'] = '637N, 637S'
# ee.loc[ee[(ee.possible_stops == '') & (ee.station_name == 'Jackson Hts - Roosevelt Av')].index,'possible_stops'] = 'G14N, G14S'
ee.loc[ee[(ee.possible_stops == '') & (ee.station_name == 'Kings Highway')].index,'possible_stops'] = 'D35N, D35S'
ee.loc[ee[(ee.possible_stops == '') & (ee.station_name == 'New Utrecht')].index,'possible_stops'] = 'N04N, N04S'

In [441]:
# ee[(ee.possible_stops == '') & (ee.station_name == 'Broadway-Lafayette/Bleecker St')][['possible_stops']] = '637N, 637S'
# ee[(ee.possible_stops == '') & (ee.station_name == 'Jackson Hts - Roosevelt Av')][['possible_stops']] = 'G14N, G14S'
# ee[(ee.possible_stops == '') & (ee.station_name == 'Kings Highway')][['possible_stops']] = 'D35N, D35S'
# ee[(ee.possible_stops == '') & (ee.station_name == 'New Utrecht')][['possible_stops']] = 'N04N, N04S'

/opt/conda/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:3395: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/opt/conda/envs/py3/lib/python3.6/site-packages/pandas/core/frame.py:3367: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [46]:
ee[ee.possible_stops == '']

,station_name,from,from_type,to,to_type,line,direction,possible_stops
16,125 St,Platform pid_2,Platform,METRO_NORTH-south,Train,METRO_NORTH,south,
17,125 St,Platform pid_2,Platform,METRO_NORTH-north,Train,METRO_NORTH,north,
506,Forest Hills - 71 Av,Platform pid_92,Platform,LIRR-jamaica,Train,LIRR,jamaica,
514,Forest Hills - 71 Av,Platform pid_93,Platform,LIRR-manhattan,Train,LIRR,manhattan,
727,Sutphin Blvd - Archer Av - JFK Airport,Platform pid_131,Platform,LIRR-south,Train,LIRR,south,
728,Sutphin Blvd - Archer Av - JFK Airport,Platform pid_131,Platform,LIRR-north,Train,LIRR,north,
744,Times Sq - 42 St,Platform pid_132,Platform,S-north,Train,S,north,
753,Times Sq - 42 St,Platform pid_132,Platform,S-south,Train,S,south,


In [47]:
ee.head()

,station_name,from,from_type,to,to_type,line,direction,possible_stops
8,125 St,Platform pid_1,Platform,A-south,Train,A,south,"A15N, A15S"
9,125 St,Platform pid_1,Platform,D-south,Train,D,south,"A15N, A15S"
10,125 St,Platform pid_1,Platform,B-south,Train,B,south,"A15N, A15S"
11,125 St,Platform pid_1,Platform,C-south,Train,C,south,"A15N, A15S"
13,125 St,Platform pid_2,Platform,5-south,Train,5,south,"621N, 621S"


In [301]:
# pattearn = re.compile('.*N')
# [pattern.match(x) for x in ee.stop_ids[0].split(',')]
# [re.findall('N',x) for x in ee.stop_ids[0].split(',')]

[<_sre.SRE_Match object; span=(0, 4), match='110N'>, None]

In [49]:
weekday[weekday.stop_id.str.startswith('7')].sort_values('stop_id').head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,line,route_id
83598,AFA19GEN-7060-Weekday-00_011650_7..N97R,02:32:00,02:32:00,701N,22,NaN,0,0,NaN,7,7
83884,AFA19GEN-7060-Weekday-00_034700_7..N97R,06:23:00,06:23:00,701N,22,NaN,0,0,NaN,7,7
83862,AFA19GEN-7060-Weekday-00_033700_7..N97R,06:12:30,06:12:30,701N,22,NaN,0,0,NaN,7,7
87538,AFA19GEN-7060-Weekday-00_139550_7..N97R,23:52:30,23:52:30,701N,22,NaN,0,0,NaN,7,7
90034,AFA19GEN-7060-Weekday-00_115850_7..N97R,19:56:30,19:56:30,701N,22,NaN,0,0,NaN,7,7


In [490]:
stops_new[stops_new.stop_id.str.contains('701')]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
465,701,NaN,Flushing - Main St,NaN,40.7596,-73.83003,NaN,NaN,1,NaN
466,701N,NaN,Flushing - Main St,NaN,40.7596,-73.83003,NaN,NaN,0,701
467,701S,NaN,Flushing - Main St,NaN,40.7596,-73.83003,NaN,NaN,0,701


In [500]:
stops_new[stops_new.stop_name.str.contains('Wood')]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
282,401,NaN,Woodlawn,NaN,40.886037,-73.878751,NaN,NaN,1,NaN
283,401N,NaN,Woodlawn,NaN,40.886037,-73.878751,NaN,NaN,0,401
284,401S,NaN,Woodlawn,NaN,40.886037,-73.878751,NaN,NaN,0,401
492,712,NaN,Woodside - 61 St,NaN,40.745630,-73.902984,NaN,NaN,1,NaN
493,712N,NaN,Woodside - 61 St,NaN,40.745630,-73.902984,NaN,NaN,0,712
494,712S,NaN,Woodside - 61 St,NaN,40.745630,-73.902984,NaN,NaN,0,712
978,G11,NaN,Woodhaven Blvd,NaN,40.733106,-73.869229,NaN,NaN,1,NaN
979,G11N,NaN,Woodhaven Blvd,NaN,40.733106,-73.869229,NaN,NaN,0,G11
980,G11S,NaN,Woodhaven Blvd,NaN,40.733106,-73.869229,NaN,NaN,0,G11
1098,J15,NaN,Woodhaven Blvd,NaN,40.693879,-73.851576,NaN,NaN,1,NaN


In [50]:
single_trips = weekday.drop_duplicates(['stop_id'],keep='first')

In [52]:
single_trips[single_trips.stop_id.str.startswith('7')].sort_values('stop_id').head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,line,route_id
83532,AFA19GEN-7060-Weekday-00_006150_7..N97R,01:37:00,01:37:00,701N,22,NaN,0,0,NaN,7,7
78167,AFA19GEN-7060-Weekday-00_001150_7..S97R,00:11:30,00:11:30,701S,1,NaN,0,0,NaN,7,7
83531,AFA19GEN-7060-Weekday-00_006150_7..N97R,01:33:30,01:33:30,702N,21,NaN,0,0,NaN,7,7
78168,AFA19GEN-7060-Weekday-00_001150_7..S97R,00:14:00,00:14:00,702S,2,NaN,0,0,NaN,7,7
83530,AFA19GEN-7060-Weekday-00_006150_7..N97R,01:31:30,01:31:30,705N,20,NaN,0,0,NaN,7,7


In [53]:
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_000600_1..S03R,South Ferry,1,NaN,1..S03R
1,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_002600_1..S03R,South Ferry,1,NaN,1..S03R
2,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_004600_1..S03R,South Ferry,1,NaN,1..S03R
3,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_006600_1..S03R,South Ferry,1,NaN,1..S03R
4,1,AFA19GEN-1037-Sunday-00,AFA19GEN-1037-Sunday-00_007200_1..N03R,Van Cortlandt Park - 242 St,0,NaN,1..N03R


In [54]:
stops_new[stops_new.location_type == 0].shape

(1002, 10)

In [55]:
stop_times.shape

(541811, 10)

In [56]:
north_bound = ['north','norwood 205 st', 'woodlawn','jamaica', 'flushing main st','jamaica center','flushing', 'wakefield 241 st','jamaica, forest hills']
south_bound = ['south','bay ridge', 'coney island','flatbush av brooklyn college','brighton beach, coney island, bay ridge','brighton beach','coney island, brighton beach','far rockaway']

def get_dir(df):
    if df.possible_stops == '': return "NULL"
    if len(df.possible_stops.split(', ')) == 1: return df.possible_stops
    # hack for Kings Highway Q line
    if len(df.possible_stops.split(', ')) == 3: return 'N08N'
    
    if df.direction in north_bound:
        match = [x for x in df.possible_stops.split(', ') if re.match('.*N$',x)]
        return match[0]
    elif (df.direction in south_bound) | ((df.direction == 'manhattan') & (df.line in ['4','6','7','Z'])):
        match = [x for x in df.possible_stops.split(', ') if re.match('.*S$',x)]
        return match[0]
    elif (df.direction == 'manhattan') & (df.line in ['3','G']):
        match = [x for x in df.possible_stops.split(', ') if re.match('.*N$',x)]
        return match[0]
    else:
        return "NULL"
        

In [57]:
tmp = '110N, 110S'
[x for x in tmp.split(', ') if re.match('.*N$',x)]

['110N']

In [58]:
tmp = 'H01N'
tmp.split(', ')

['H01N']

In [59]:
ee['stop_id'] = ee.apply(get_dir,axis=1)

In [60]:
ee.direction.unique()

array(['south', 'north', 'manhattan', 'norwood 205 st', 'woodlawn',
       'jamaica', 'flushing main st', 'coney island',
       'flatbush av brooklyn college', 'jamaica center', 'flushing',
       'wakefield 241 st', 'coney island, brighton beach',
       'jamaica, forest hills', 'far rockaway', 'brigton beach',
       'brighton beach', 'east', 'west',
       'brighton beach, coney island, bay ridge'], dtype=object)

In [64]:
ee_tmp = ee.copy()

In [65]:
ee.loc[(ee.stop_id == 'NULL') & ~(ee.possible_stops == ''),'stop_id'] = ['221S','221S','D11S','D11S','F01N','B04S','235N','B21N','F27N',
 '244N','244N','M12S','M12S','G08S','G08S','G08S','G08S','F06S','F06S',
 'D35N','M16S','M16S','504S','211S','D26N','D26N','G21S','G21S','G21S',
 '217S','217S','D35S','725N','725S','R30N','R30N','R30N']

In [66]:
ee[(ee.stop_id == "NULL") & (ee.direction.str.contains('man'))].line.unique()

array(['LIRR'], dtype=object)

In [68]:
ee[(ee.stop_id == "NULL")].head()

,station_name,from,from_type,to,to_type,line,direction,possible_stops,stop_id
16,125 St,Platform pid_2,Platform,METRO_NORTH-south,Train,METRO_NORTH,south,,NULL
17,125 St,Platform pid_2,Platform,METRO_NORTH-north,Train,METRO_NORTH,north,,NULL
506,Forest Hills - 71 Av,Platform pid_92,Platform,LIRR-jamaica,Train,LIRR,jamaica,,NULL
514,Forest Hills - 71 Av,Platform pid_93,Platform,LIRR-manhattan,Train,LIRR,manhattan,,NULL
727,Sutphin Blvd - Archer Av - JFK Airport,Platform pid_131,Platform,LIRR-south,Train,LIRR,south,,NULL


In [71]:
plt_stop_id = ee[['station_name','from','line','direction','stop_id']]
plt_stop_id = plt_stop_id[plt_stop_id.stop_id != 'NULL']
plt_stop_id['platform_id'] = [x.split(' ')[1] for x in plt_stop_id['from']]

In [74]:
plt_stop_id = plt_stop_id[['station_name','platform_id','line','direction','stop_id']]
plt_stop_id.head()

,station_name,platform_id,line,direction,stop_id
8,125 St,pid_1,A,south,A15S
9,125 St,pid_1,D,south,A15S
10,125 St,pid_1,B,south,A15S
11,125 St,pid_1,C,south,A15S
13,125 St,pid_2,5,south,621S


In [75]:
plt_stop_id.to_csv('../../data/processed/platform_id_to_GTFS_mapping.csv')

In [353]:
ee_w_stops = pd.read_csv('../../data/processed/elevator_to_GTFS_KM.csv')

In [355]:
ee_w_stops.shape

(557, 8)

In [357]:
ee_w_stops[ee_w_stops.stop_id.isnull()]

,Unnamed: 0,Unnamed: 0.1,equipment_id,line,direction,station,possible_stops,stop_id
2,2,185,EL122,6,manhattan,Pelham Bay Park,"601N, 601S",NaN
7,7,1464,EL126,METRO-NORTH,south,125 St,NaN,NaN
8,8,1255,EL126,METRO-NORTH,north,125 St,NaN,NaN
12,12,404,EL127,5,wakefield 241 st,Simpson St,"217N, 217S",NaN
13,13,195,EL127,2,wakefield 241 st,Simpson St,"217N, 217S",NaN
14,14,403,EL128,5,manhattan,Simpson St,"217N, 217S",NaN
15,15,194,EL128,2,manhattan,Simpson St,"217N, 217S",NaN
18,18,220,EL130,5,manhattan,149 St - 3 Av,"221N, 221S",NaN
19,19,11,EL130,2,manhattan,149 St - 3 Av,"221N, 221S",NaN
20,20,15,EL132,4,woodlawn,161 St - Yankee Stadium,"414N, 414S",NaN
